In [35]:
import os
import copy
import torch
import random
import numpy as np
from torch import nn
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

from mmdet.models.roi_heads import SingleRoIExtractor




In [36]:
def get_model_outputs(model: nn.Module, func_name: str,
                      model_inputs: dict) -> Any:
    """To get outputs of pytorch model.

    Args:
        model (nn.Module): The input model.
        func_name (str): The function of model.
        model_inputs (dict): The inputs for model.

    Returns:
        Any: The output of model, decided by the input model.
    """
    assert hasattr(model, func_name), f'Got unexpected func name: {func_name}'
    func = getattr(model, func_name)
    model_outputs = func(**copy.deepcopy(model_inputs))
    return model_outputs

def seed_everything(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.enabled = False



In [37]:
def get_single_roi_extractor():
    """SingleRoIExtractor Config."""
    roi_layer = dict(type='RoIAlign', output_size=7, sampling_ratio=2)
    out_channels = 1
    featmap_strides = [4, 8, 16, 32]
    model = SingleRoIExtractor(roi_layer, out_channels, featmap_strides).eval()

    return model

In [60]:
single_roi_extractor = get_single_roi_extractor()

seed_everything(1234)
out_channels = single_roi_extractor.out_channels
feats = [
    torch.rand((1, out_channels, 200, 336)),
    torch.rand((1, out_channels, 100, 168)),
    torch.rand((1, out_channels, 50, 84)),
    torch.rand((1, out_channels, 25, 42)),
]
seed_everything(5678)
rois = torch.tensor([[0.0000, 587.8285, 52.1405, 886.2484, 341.5644]])

model_inputs = {
    'feats': feats,
    'rois': rois,
}
model_outputs = get_model_outputs(single_roi_extractor, 'forward',
                                  model_inputs)
model_outputs

tensor([[[[0.3466, 0.5128, 0.5111, 0.5483, 0.5056, 0.5312, 0.6108],
          [0.5050, 0.4760, 0.6118, 0.3697, 0.3654, 0.3364, 0.3462],
          [0.5681, 0.6063, 0.7787, 0.4521, 0.3313, 0.4873, 0.4783],
          [0.6419, 0.6106, 0.6516, 0.5381, 0.4222, 0.5601, 0.4479],
          [0.6377, 0.7092, 0.5607, 0.5814, 0.3036, 0.5156, 0.3908],
          [0.4553, 0.5963, 0.6055, 0.5186, 0.3104, 0.5165, 0.3619],
          [0.4918, 0.6542, 0.5296, 0.5941, 0.3207, 0.4624, 0.4704]]]])